In [1]:
import os, errno
import argparse
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from scipy.stats import kurtosis
from scipy.stats.mstats import moment
from scipy import stats
from scipy.stats import ttest_ind, f_oneway

from util.load_data_basic import *

# date_time format
date_time_format = '%Y-%m-%dT%H:%M:%S.%f'
date_only_date_time_format = '%Y-%m-%d'

# sleep after work duration thereshold
sleep_after_work_duration_threshold = 12
sleep_after_sleep_duration_threshold = 2

# data folder
main_data_directory = '../data/keck_wave1/2_preprocessed_data'
recording_timeline_directory = '../output/recording_timeline'
sleep_timeline_directory = '../output/sleep_timeline'
individual_timeline_directory = 'output/individual_timeline'



## Read Basic Information

In [2]:
# Read sleep data for all participant
data = pd.read_csv(os.path.join('output', 'sleep_survey_full.csv'))

# Read ID
IDs = getParticipantID(main_data_directory, index=False)

# Read MGT
MGT = read_MGT(main_data_directory)

# Read Pre-Study info
PreStudyInfo = read_pre_study_info(main_data_directory)

# Read IGTB info
IGTB = read_IGTB(main_data_directory)

# Day shift data
day_data = data.loc[data['shift_type'] == 1]
day_workday_data = day_data.loc[(day_data['is_sleep_before_work'] == 1) & (day_data['is_sleep_after_work'] == 1)]
day_off_day_data = day_data.loc[(day_data['is_sleep_before_work'] != 1) & (day_data['is_sleep_after_work'] != 1)]
day_transition_day_data = day_data.loc[(day_data['is_sleep_transition_before_work'] == 1) | (day_data['is_sleep_transition_after_work'] == 1)]

# Night shift data
night_data = data.loc[data['shift_type'] == 2]
night_workday_data = night_data.loc[(night_data['is_sleep_before_work'] == 1) & (night_data['is_sleep_after_work'] == 1)]
night_off_day_data = night_data.loc[(night_data['is_sleep_before_work'] != 1) & (night_data['is_sleep_after_work'] != 1)]
night_transition_day_data = night_data.loc[(night_data['is_sleep_transition_before_work'] == 1) | (night_data['is_sleep_transition_after_work'] == 1)]

# Data array
data_array = [day_data, night_data, 
              day_workday_data, day_off_day_data, 
              night_workday_data, night_off_day_data]

data_type = ['day-shift all', 'night-shift all',
             'day-shift workday', 'day-shift off day', 
             'night-shift workday', 'night-shift off day']

colunm_type = ['duration_in_seconds', 'SleepEfficiency',
               'SleepMinutesStageDeep', 'SleepMinutesStageLight', 'SleepMinutesStageRem',
               'sleep_heart_rate_mean', 'sleep_heart_rate_std',
               'sleep_heart_rate_percentile_10', 'sleep_heart_rate_percentile_90']

UserInfo = pd.merge(IGTB, PreStudyInfo, left_on='uid', right_on='uid', how='outer')
UserInfo = pd.merge(UserInfo, IDs, left_on='uid', right_on='uid', how='outer')



## Plot MGT

In [39]:
min_sleep = np.min(MGT['sleep_mgt'].dropna())
max_sleep = np.max(MGT['sleep_mgt'].dropna())
sleep_range = max_sleep - min_sleep

min_pos_af = np.min(MGT['pos_af_mgt'].dropna())
max_pos_af = np.max(MGT['pos_af_mgt'].dropna())
pos_af_range = max_pos_af - min_pos_af

for user_id in IDs['uid']:
    participant_id = IDs.loc[IDs['uid'] == user_id]['participant_id'].values[0]
    shift = IDs.loc[IDs['uid'] == user_id]['shift'].values[0]
    
    participant_MGT = MGT.loc[MGT['uid'] == user_id]
    
    # survey type
    participant_MGT_health = participant_MGT.loc[participant_MGT['survey_type'] == 'health']
    participant_MGT_job = participant_MGT.loc[participant_MGT['survey_type'] == 'job']
    participant_MGT_personality = participant_MGT.loc[participant_MGT['survey_type'] == 'personality']
    
    if len(participant_MGT_health['sleep_mgt']) > 10:
        
        sleep_mgt = participant_MGT_health[['sleep_mgt', 'pos_af_mgt']].dropna()
        sleep_mgt_date_value = [pd.to_datetime(date) for date in sleep_mgt.index.values]
        
        pos_mgt = participant_MGT['pos_af_mgt'].dropna()
        pos_mgt_date_value = [pd.to_datetime(date) for date in pos_mgt.index.values]
        
        # Stats
        sleep_mgt_corr = sleep_mgt.dropna().corr(method='pearson')
        
        print('sleep vs pos_af')
        # print('Correlation = %.3f \n' % sleep_mgt_corr['sleep_mgt']['pos_af_mgt'])
        print(sleep_mgt_corr)
        
        # if participant_id == '757d34ef-7f1b-41f5-b072-ea4a4f336b80':
        #    print(sleep_mgt)
        
        # Plot
        if np.abs(sleep_mgt_corr['sleep_mgt']['pos_af_mgt']) > 0.8:
            fig = plt.figure()
            ax = plt.axes()

            ax.plot(sleep_mgt_date_value, (sleep_mgt['sleep_mgt'].values - min_sleep) / sleep_range)
            # ax.plot(pos_mgt_date_value, (pos_mgt.values - min_pos_af) / pos_af_range)
            ax.plot(sleep_mgt_date_value, (sleep_mgt['pos_af_mgt'].values - min_pos_af) / pos_af_range)

            ax.set_xlim([pd.to_datetime(participant_MGT.index.values[0]), pd.to_datetime(participant_MGT.index.values[-1])])
            ax.set_ylim([0, 1])
            plt.xticks(rotation=90)
            plt.show()
        
        
        # Debug
        # print(sleep_mgt.values)
    
    

sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000    0.011289
pos_af_mgt   0.011289    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000   -0.266497
pos_af_mgt  -0.266497    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000    0.069028
pos_af_mgt   0.069028    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000    0.070328
pos_af_mgt   0.070328    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000   -0.353449
pos_af_mgt  -0.353449    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000    0.266007
pos_af_mgt   0.266007    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000   -0.153027
pos_af_mgt  -0.153027    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_mgt    1.000000   -0.253272
pos_af_mgt  -0.253272    1.000000
sleep vs pos_af
            sleep_mgt  pos_af_mgt
sleep_